In [ ]:
import numpy as np
import pandas as pd
from scipy.special import logit
import lightgbm as lgb

# Load the train and test datasets
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train = train.drop('ID_code', axis = 1)

test_id = test.ID_code
test = test.drop('ID_code', axis = 1)

correlations = train.drop("target", axis = 1).corr().abs().unstack().sort_values(kind = "quicksort").reset_index()
correlations = correlations[correlations['level_0'] != correlations['level_1']]

variables = train.drop("target", axis = 1).columns.values.tolist()
corr_pre_res = np.zeros(len(variables))
i = 0
for var in variables:
    corr_pre_res[i] = np.corrcoef(train[var], train["target"])[0, 1]
    i += 1

corr_pre_res = abs(pd.DataFrame(corr_pre_res))
corr_pre_res.columns = ['corr_pre_res']
corr_pre_res.sort_values(by = 'corr_pre_res')

# Define the features list (assuming that features start with 'var')
features = [x for x in train.columns if x.startswith("var")]

# Create the hist_df DataFrame for value counts
hist_df = pd.DataFrame()

# Loop through each feature to calculate value counts
for var in features:
    # Concatenate the 'train' and 'test' series for the current feature
    var_stats = pd.concat([train[var], test[var]]).value_counts()  # Use pd.concat() instead of append()

    # Map the value counts to the 'test' data for the current variable
    hist_df[var] = test[var].map(var_stats)

    # Set values to 1 if the count of that value is greater than 1
    hist_df[var] = hist_df[var] > 1

# Continue with your model training and predictions
ind = hist_df.sum(axis=1) != 200
var_stats = {var: pd.concat([train[var], test[ind][var]]).value_counts() for var in features}

pred = 0
for var in features:
    model = lgb.LGBMClassifier(**{'learning_rate': 0.05,
                                  'max_bin': 165,
                                  'max_depth': 5,
                                  'min_child_samples': 150,
                                  'min_child_weight': 0.1,
                                  'min_split_gain': 0.0018,
                                  'n_estimators': 41,
                                  'num_leaves': 6,
                                  'reg_alpha': 2.0,
                                  'reg_lambda': 2.54,
                                  'objective': 'binary',
                                  'n_jobs': -1})
    model = model.fit(np.hstack([train[var].values.reshape(-1, 1),
                      train[var].map(var_stats[var]).values.reshape(-1, 1)]), train["target"].values)
    pred += logit(model.predict_proba(np.hstack([test[var].values.reshape(-1, 1),
                  test[var].map(var_stats[var]).values.reshape(-1, 1)]))[:, 1])

pd.DataFrame({"ID_code": test_id, "target": pred}).to_csv("Submission.csv", index = False)

C:\Users\Natha\AppData\Local\Temp\ipykernel_2952\2729243670.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  hist_df[var] = test[var].map(var_stats)
C:\Users\Natha\AppData\Local\Temp\ipykernel_2952\2729243670.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  hist_df[var] = test[var].map(var_stats)
C:\Users\Natha\AppData\Local\Temp\ipykernel_2952\2729243670.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joini

[LightGBM] [Info] Number of positive: 20098, number of negative: 179902
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000516 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 180
[LightGBM] [Info] Number of data points in the train set: 200000, number of used features: 2
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.100490 -> initscore=-2.191792
[LightGBM] [Info] Start training from score -2.191792
[LightGBM] [Info] Number of positive: 20098, number of negative: 179902
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000557 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 178
[LightGBM] [Info] Number of data points in the train set: 200000, number of used features: 2
[LightGBM] [Info] 